# Simulation

Each simulation is encapsulated as an object that is responsible for
finding the location of each dataset in the simulation, loading it —
which may involve downloading and caching the relevant file — and
returning an object that can be used to access that piece of data.


## Location

We begin by simply loading the simulation.  There are one required and
two optional pieces of information needed to identify a simulation:

  1. SXS ID

     This identifies the simulation type and includes a number.  For
     example, "SXS:BBH:0001", which identifies this simulation as a
     product of the SXS collaboration, specifies that it is a binary
     black hole simulation, and that it has been assigned the number
     "0001" in that series.  Note that these numbers are not
     necessarily sequential.  Nor do they necessarily correspond to
     the relative age of the simulation; that information is contained
     in the metadata itself in the various `date_*` keys.

  2. Version
     
     This identifies the version of the data — like "v3.0".  Unlike
     the SXS ID, this is an optional specifier.  If not provided, the
     most recent version is used.  All versions refer to the same
     underlying simulation, but the raw data may have been processed
     differently, may be provided in incompatible formats, etc.  For
     exploratory work, it is often convenient to simply use the most
     recent version.  However, for reproducibility, it is important to
     specify the version of the data you use for a given analysis.

   3. Lev (resolution)
      
      This identifies the resolution of the simulation — like "Lev5".
      This is also optional.  If not provided, the highest resolution
      is used.  Note that there is no consistency in the "Lev"s
      provided for different simulations, nor is there necessarily
      even consistency in the meaning of the "Lev" between different
      simulations (they are not always directly comparable).  Again,
      for reproducibility, it is important to specify the resolution
      of the data you use for a given analysis.

These three pieces of information may be combined into a single string as in
any of the following examples of valid inputs:

    SXS:BBH:0001
    SXS:BBH:0001v2.0
    SXS:BBH:0001/Lev5
    SXS:BBH:0001v2.0/Lev5

The full specification including ID, version, and Lev is called the
"location", but any of these can be provided to load the simulation:

```python
sxs.load("SXS:BBH:0001")
sxs.load("SXS:BBH:0001v2.0")
sxs.load("SXS:BBH:0001/Lev5")
sxs.load("SXS:BBH:0001v2.0/Lev5")
```

## Deprecated or superseded

Many simulations are now quite old, and do not have the benefit of
years of refinements to the simulation code.  As a result, the SXS
collaboration has deprecated many of them, and replaced them with
newer simulations of (nearly) the same physical parameters.

By default, a deprecated simulation raise an error if you attempt to
load it.  However, you can still load it if you want to, by choosing
one of the following options:

  1. Pass `ignore_deprecation=True` to load the latest available
     version.  This completely bypasses even checking for deprecation or
     supersession.  No warnings or errors will be issued.
  2. Manually choose a different simulation from the catalog.
  3. Pass `auto_supersede=0.01` to load a match "closer" than 0.01 in
     the catalog if one exists; and error otherwise.  Here,
     "closeness" is measured in terms of a simplistic metric on the
     metadata — specifically the basic information like masses, spins,
     and eccentricity measured at the reference time.  This is
     imperfect, because it ignores the evolution of these quantities,
     which may differ for different simulations even when they
     represent the same physical system.
  4. Pass `auto_supersede=True` to load the "closest" match in the
     catalog, even if it is quite "far" away.
  5. Include the version number, as in 'SXS:BBH:0001v2.0', to load a
     specific version.  A warning will be issued that the simulation is
     deprecated, but it will be loaded anyway.

In this case, "SXS:BBH:0001" is deprecated, and has been superseded by
the newer simulation "SXS:BBH:1132".  We can automatically load the
superseding simulation with

In [1]:
import sxs

sim = sxs.load("SXS:BBH:0001", auto_supersede=True)

Loading SXS simulations using latest tag 'v3.0.0a10', published at 2025-04-23T15:08:30Z.


/Users/boyle/.continuum/current/envs/science/lib/python3.12/site-packages/sxs/simulations/simulation.py:236: UserWarning: 
Simulation 'SXS:BBH:0001v2.0' is being automatically superseded by 'SXS:BBH:1132'.
The distance between them in the given metadata metric is 3.33e-07.
  warn(message)


Note that, because we did not load a tagged version of the simulations
catalog [with, e.g., `sxs.load("dataframe", tag="3.0.0")`] before
loading the waveform, `sxs` just found the most recent tag and loaded
that version of the catalog, then used the information there to find
which simulation to automatically supersede with.

We can see that, even though we requested "SXS:BBH:0001", the location
of the output simulation object is "SXS:BBH:1132".  A warning is
printed, even though we asked for the superseding simulation, so that
it is clear which  simulation we are actually using, though we can
also access this programmatically:

In [2]:
sim.location

'SXS:BBH:1132v3.0/Lev4'

Note that the version "v3.0" and "Lev4" were automatically chosen as the highest values, respectively.

## Metadata

Having loaded the simulation, the only thing loaded so far is the
metadata about the simulation.  This is encapsulated in the `metadata`
attribute of the simulation object.  This is a dictionary-like object
that contains all of the metadata about the simulation, including the
SXS ID, version, and Lev, as well as the masses and spins of the black
holes, the reference time, and other important information, such as
where to find the various types of data for this simulation.  These
fields are described in detail in [the most recent SXS catalog
paper](https://doi.org/10.26138/sxscatalog3), and are used in
constructing the catalog dataframe described on [the previous
page](/tutorials/01-Catalog_Dataframe).

We can see its contents here:

In [3]:
sim.metadata

Metadata([('simulation_name', 'BBH_SKS_d23.1_q1_sA_0_0_0_sB_0_0_0/Lev4'),
          ('keywords', []),
          ('alternative_names', ['PRIVATE:BBH:0143', 'SXS:BBH:1132']),
          ('metadata_format_revision', 2),
          ('metadata_content_revision', 1),
          ('internal_minor_version', 0),
          ('internal_changelog', {}),
          ('citation_dois', ['10.1088/1361-6382/ab34e2']),
          ('initial_separation', 23.12716512),
          ('initial_orbital_frequency', 0.00851783),
          ('initial_adot', -4.675904e-06),
          ('initial_ADM_energy', 0.994987997679003),
          ('initial_ADM_linear_momentum',
           [-2.156564e-10, 3.97011e-10, 4.51632e-11]),
          ('initial_ADM_angular_momentum',
           [3.54751846e-08, 1.445901368e-07, 1.3091640743018833]),
          ('initial_dimensionless_spin1',
           [1.2758e-12, -3.8937e-12, -6.57154e-10]),
          ('initial_dimensionless_spin2',
           [-3.554e-13, 9.477e-13, -3.283941e-10]),
          

However, one important point to note about the metadata is that it is
not always in a format we might expect.  For example, the
`reference_eccentricity` field will often be a string like
`"<7.2e-05"`, which makes it awkward to compute with.  These sorts of
issues are avoided with the `dataframe` interface described on the
previous page by forcing fields to have consistent types.

We can extract the metadata for this one simulation in a format consistent with other simulations as a `pandas.Series` object with the attribute `sim.series`.  (Here, we'll use `pandas` explicitly to display the series with a little more detail.)

In [4]:
import pandas as pd
with pd.option_context("max_colwidth", 46, "display.min_rows", 30):
    display(sim.series)

deprecated                                                                      False
reference_mass_ratio                                                              1.0
reference_chi_eff                                                                -0.0
reference_chi1_perp                                                               0.0
reference_chi2_perp                                                               0.0
reference_eccentricity                                                       0.000027
reference_eccentricity_bound                                                 0.000027
reference_time                                                                  170.0
reference_dimensionless_spin1_x                                                   0.0
reference_dimensionless_spin1_y                                                  -0.0
reference_dimensionless_spin1_z                                                  -0.0
reference_dimensionless_spin1_mag                     

Various relevant pieces of information about the simulation are also
available as attributes of the simulation object.  For example, we may
wish to know which versions are available:

In [5]:
sim.versions

['', 'v1.2', 'v1.3', 'v1.4', 'v1.5', 'v2.0', 'v3.0']

In this case, the generic version, `''`, as well as multiple older
versions from the first ('v1.x'), second ('v2.0'), and latest ('v3.0')
versions of the catalog are available, because SXS:BBH:1132 is a
fairly old simulation.

These versions track modifications to the files representing the data
and — together with the SXS ID — establish the unique identifier for
the data set.  This unique identifier is also published as a DOI.  The
DOI prefix for SXS data is 10.26138, and the full DOI for any
simulation is given by combining these:

In [6]:
sim.url

'https://doi.org/10.26138/SXS:BBH:1132v3.0'

These DOIs are permanent and can be used to refer to the data in
publications, in the same way that DOIs for journal articles are used.
They point to deposits of the data in the Zenodo or CaltechDATA
repository, which are long-term, open-access archives.  This is also
where the data are automatically obtained when you load a particular
data set for the first time.

## Data

Besides the metadata, the remaining data sets are loaded lazily.  This
means that we can access the data as needed, but the cost in time and
resources is not paid unless and until the data is actually accessed.
Specifically, the time to download the data if needed, the disk space
required to cache it if desired, the time to load the data from disk,
and the memory required to store it are all deferred to the point of
use.

For example, we can access data describing the horizons as

In [7]:
sim.horizons

And data describing the waveform as

In [8]:
sim.h

WaveformModes([[-3.84045085e-02-5.89198155e-03j, -5.77778752e-08+5.89152049e-07j,
        -2.04127826e-03-7.59793532e-09j, ...,
        -1.69433081e-05+1.02668263e-04j, -5.52232115e-09+3.01949886e-08j,
        -1.36598721e-07-1.89939464e-04j],
       [-4.08725041e-02-6.17793871e-03j, -1.12721753e-07+1.22522360e-06j,
        -2.61140490e-03+2.24617634e-08j, ...,
         1.58363235e-04-1.07098592e-04j,  1.88905886e-08+8.79536268e-09j,
        -2.16597212e-04+1.81872383e-04j],
       [-4.41104676e-02-6.91220296e-03j,  6.66523849e-08+4.93630433e-07j,
        -2.35699925e-03+3.70045888e-08j, ...,
         1.61960978e-04-2.16341762e-04j, -2.32333702e-08-9.77603142e-08j,
        -1.86282822e-04+3.95645415e-04j],
       ...,
       [-1.34434621e-04-2.50412266e-04j,  2.01020405e-10-1.69444925e-10j,
         8.78545565e-02+3.45608094e-11j, ...,
         1.69202525e-07+2.06979377e-08j, -6.86001843e-11+5.93489900e-11j,
         8.13201650e-07-8.93233959e-08j],
       [-1.34415470e-04-2.50381871e-

The objects returned will be the subject of the next two notebooks in this series:

- [`Horizons`](/tutorials/03-Horizons)
- [`WaveformModes`](/tutorials/04-Waveforms)